In [1]:
import app
instance = app.create_app()
instance.app_context().push()

d:\dailydiet\dailydiet-api\venv\lib\site-packages\whitenoise\base.py:115: UserWarning: No directory at: D:\dailydiet\dailydiet-api\admin\static\
  warnings.warn(u"No directory at: {}".format(root))


In [2]:
from foods.models import Food,DietRecord
from extentions import elastic

In [4]:
page = Food.query.paginate(page=1,per_page=100)

In [ ]:
page.next()

In [5]:
page.pages

60

In [9]:
page = Food.query.paginate(page=26,per_page=100)
while page.has_next:
    for item in page.items:
        Food.add_to_index(item)
    print(f'page {page.next_num - 1} indexed.')
    page = page.next()

page 26 indexed.
page 27 indexed.
page 28 indexed.
page 29 indexed.
page 30 indexed.
page 31 indexed.
page 32 indexed.
page 33 indexed.
page 34 indexed.
page 35 indexed.
page 36 indexed.
page 37 indexed.
page 38 indexed.
page 39 indexed.
page 40 indexed.
page 41 indexed.
page 42 indexed.
page 43 indexed.
page 44 indexed.
page 45 indexed.
page 46 indexed.
page 47 indexed.
page 48 indexed.
page 49 indexed.
page 50 indexed.
page 51 indexed.
page 52 indexed.
page 53 indexed.
page 54 indexed.
page 55 indexed.
page 56 indexed.
page 57 indexed.
page 58 indexed.
page 59 indexed.


In [ ]:
page = Food.query.paginate(page=51,per_page=100)

In [10]:
#inserting last page
page = Food.query.paginate(page=60,per_page=100)
for item in page.items:
    Food.add_to_index(item)
print(f'page {page.pages} indexed.')

page 60 indexed.


In [106]:
query = "Mangos" 
#elastic ingredient search
search = elastic.search(
            index='ingredients',
            body={'query': {'multi_match': {'query': query, 'fields': ['food_name']}},
                  'from': 0, 'size': 20})


In [107]:
search

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 7.2770195,
  'hits': [{'_index': 'ingredients',
    '_type': 'ingredient',
    '_id': '164513',
    '_score': 7.2770195,
    '_source': {'food_name': 'Mangos',
     'id': 164513,
     'primary_thumbnail': 'https://images.eatthismuch.com/site_media/thmb/164513_simmyras_016d1a64-02af-4910-bfe4-184e12b138a9.png',
     'nutrition': {'calories': 130.0,
      'carbs': 32.0,
      'fats': 0.0,
      'proteins': 1.0}}},
   {'_index': 'ingredients',
    '_type': 'ingredient',
    '_id': '1448',
    '_score': 7.2770195,
    '_source': {'food_name': 'Mangos',
     'id': 1448,
     'primary_thumbnail': 'https://images.eatthismuch.com/site_media/thmb/1448_erin_m_80d7aa63-91c7-4999-9942-d6ebea57bb2a.png',
     'nutrition': {'calories': 124.2,
      'carbs': 31.01,
      'fats': 0.79,
      'proteins': 1.7}}}]}}

In [8]:
query = "pasta olive"
# search = elastic.search(
#             index='foods_new',
#             body={
#   "explain": True,
#   "query": {
#     "bool": {
#       "should": [
#         {
#           "multi_match": {
#             "query": query,
#             "fields": [
#               "name^4.0",fr
#               "category^1.0",
#               "description^3.0"
#               "tag_cloud^4.0",
#               "ingredients^2.0",
#               "directions^1.5",
#               "author^1.0"
#             ],
#             "type": "phrase_prefix",
#             "lenient": "true"
#           }
#         }
#       ],
#       "boost": 1,
#       "minimum_should_match": 1
#     }
#   },
#   "from": 0,
#   "size": 20}
# )
q = Q('bool',
#          should=[
#             {
#               "multi_match": {
#                   "query": query,
#                   "fields": [
#                       "name^6.0",
#                       "category^1.0",
#                       "description^3.0",
#                       "tag_cloud^3.0",
#                       "ingredients^2.0",
#                       "directions^1.5",
#                       "author^1.0"
#                 ],
#                 "type": "phrase_prefix",
#                 "lenient": "true"
#               }
#             },
             
#          ],
      must=[
             {
                 "range":{
                     "nutrition.calories":{
                         "gte":2800
                     }
                 }
             }
#           ,{
#                  "range":{
#                      "prep_time":{
#                          "gte":15
#                      }
#                  }
#              }
         ],boost=1,
      minimum_should_match=1)

      

In [3]:
from elasticsearch_dsl import Q,Search


In [9]:
s = Search(using=elastic)
s = s.query(q)
for idd in [4914 ,2057,2042]:
    s = s.filter('term',**{
               "ingredient_ids": idd
            })
s[0].execute().to_dict()

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 16, 'successful': 16, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [4]:
DietRecord.query.filter(DietRecord.ownerId == 1).order_by(DietRecord.generatedAt.desc()) \
        .limit(10).all()

[]

In [14]:
query= 'roast beef'
page=1
per_page=5
search = Search(using=elastic, index='foods_new')
elastic_query = Q('bool',
                should=[
                              {
                                  "multi_match": {
                                      "query": query,
                                      "fields": [
                                          "name^6.0",
                                          "category^1.0",
                                          "description^3.0",
                                          "tag_cloud^3.0",
                                          "ingredients^2.0",
                                          "directions^1.5",
                                          "author^1.0"
                                      ],
                                      "type": "phrase_prefix",
                                      "lenient": "true"
                                  }
                              },

                          ],
                          boost=1,
                          minimum_should_match=1)
from_index = (page - 1) * per_page
size = per_page
search = search.query(elastic_query)
search_results = search[from_index: from_index + size].execute()


In [15]:
search_results.to_dict()

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 17, 'relation': 'eq'},
  'max_score': 62.20536,
  'hits': [{'_index': 'foods_new',
    '_type': '_doc',
    '_id': '33533',
    '_score': 62.20536,
    '_source': {'author': 'Ken Adams',
     'name': 'Basic roast beef',
     'description': '',
     'category': 'mostly_meat',
     'nutrition': {'calories': 129.75333333333333,
      'carbs': 0.03666666666666667,
      'fats': 9.813333333333334,
      'proteins': 9.700000000000001},
     'tag_cloud': 'Olive oil Pepper "Basic roast beef" Beef chuck Beef Products Spices and Herbs Fats and Oils Salt',
     'ingredients': ['Beef chuck', 'Olive oil', 'Salt', 'Pepper'],
     'ingredient_ids': [2899, 266, 221, 205],
     'directions': ['Preheat oven to 450F (230C). Place beef in a roasting pan, cover with a clean kitchen towel or plastic wrap and let stand at room temp for 20 mins.',
      'Using a glazing brush, br